In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rashikrahmanpritom/plant-disease-recognition-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/plant-disease-recognition-dataset


In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os

In [ ]:
/kaggle/input/chest-xray-pneumonia/chest_xray/train

In [4]:
dir=r"/kaggle/input/plant-disease-recognition-dataset/Train/Train"
os.listdir(dir)

['Healthy', 'Rust', 'Powdery']

In [5]:
classes=os.listdir(dir)

all_img_path=[]
all_img_clas=[]
for clas in classes :

    full_path = dir + "/" + clas
    img_names = os.listdir(full_path)
    img_path = [full_path + "/" + img_name for img_name in img_names]
    img_clas = [clas for img in img_path]
    all_img_path.extend(img_path)
    all_img_clas.extend(img_clas)

In [6]:
df = pd.DataFrame({
    "imags_path" : all_img_path ,
    "img_classes" : all_img_clas
})
df

,imags_path,img_classes
0,/kaggle/input/plant-disease-recognition-datase...,Healthy
1,/kaggle/input/plant-disease-recognition-datase...,Healthy
2,/kaggle/input/plant-disease-recognition-datase...,Healthy
3,/kaggle/input/plant-disease-recognition-datase...,Healthy
4,/kaggle/input/plant-disease-recognition-datase...,Healthy
...,...,...
1317,/kaggle/input/plant-disease-recognition-datase...,Powdery
1318,/kaggle/input/plant-disease-recognition-datase...,Powdery
1319,/kaggle/input/plant-disease-recognition-datase...,Powdery
1320,/kaggle/input/plant-disease-recognition-datase...,Powdery


In [7]:
df.img_classes.value_counts()

,count
img_classes,
Healthy,458
Rust,434
Powdery,430


In [8]:
import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm

In [9]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [10]:
train_ds=datasets.ImageFolder(root=dir,transform=transform)
val_ds=datasets.ImageFolder(root=dir,transform=transform)
train_loader=DataLoader(train_ds,batch_size=32,shuffle=True)
val_loader=DataLoader(val_ds,batch_size=32,shuffle=True)

In [11]:
model=models.resnet18(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 194MB/s]


In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
model.fc=nn.Linear(in_features=512,out_features=3)

In [14]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)
epouqe=7

In [15]:
for e in range(epouqe):
  model.train()
  total_loss=0
  for batch in tqdm(train_loader,total=len(train_loader)):
    imgs , labels = batch
    imgs=imgs
    labels=labels
    optimizer.zero_grad()
    outs=model(imgs)
    loss_val=loss_fn(outs,labels)
    loss_val.backward()
    optimizer.step()
    total_loss+=loss_val.item()
  print(total_loss / len(train_loader))

100%|██████████| 42/42 [05:42<00:00,  8.15s/it]


0.16138843519590973


100%|██████████| 42/42 [05:25<00:00,  7.74s/it]


0.09017058571667544


100%|██████████| 42/42 [05:24<00:00,  7.73s/it]


...

0.04808743362913707


100%|██████████| 42/42 [05:15<00:00,  7.52s/it]


0.014524127335573126


100%|██████████| 42/42 [05:16<00:00,  7.53s/it]

0.011021881209065517
